In [14]:
from selenium import webdriver
from time import time, sleep
import smtplib, ssl
from sched import scheduler
import pandas as pd



def send_email(text):
    text_dict = {'szallitasi_ido': """\Subject: Van szallitasi ido! \n \n 
        Gyorsan gyorsan!""",
                'eleszto': """\Subject: Van eleszto!! \n \n 
        Gyorsan gyorsan!"""}
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL("smtp.gmail.com", context=context) as server:
        server.login(login[0], login[1])
        server.sendmail(login[0], 'andras.hidvegi94@gmail.com', text_dict[text])
        
def get_szallitasi_ido():
    text = 'szallitasi_ido'
    van=False
    url = 'https://kifli.hu'

    login[0] = login[0].replace('\n', '')
    browser = webdriver.Chrome()
    browser.get(url)
    browser.find_element_by_id("headerLogin").click()
    browser.find_element_by_id("email").send_keys(login[0])
    browser.find_element_by_id("password").send_keys(login[1])
    browser.find_element_by_class_name('LoginBox__button').click()
    sleep(2)
    szallitasi_ido = browser.find_element_by_xpath('//*[@id="header"]/div[1]/div[1]/button/span')
    if szallitasi_ido.text == 'Nincs szállítási idő\nSzállítási idők':
        print ("Nincs szállítási idő! {}".format(pd.datetime.now()))
        browser.close()
        
    else:
        send_email(text)
        van = True
        browser.close()
    return van

def get_eleszto():
    van = False
    text = 'eleszto'
    url = 'https://kifli.hu'
    login = open('login.txt').readlines()
    login[0] = login[0].replace('\n', '')
    browser = webdriver.Chrome()
    browser.get(url)
    browser.find_element_by_id("headerLogin").click()
    browser.find_element_by_id("email").send_keys(login[0])
    browser.find_element_by_id("password").send_keys(login[1])
    browser.find_element_by_class_name('LoginBox__button').click()
    eleszto_url = 'https://kifli.hu/c300114076-eleszto'
    browser.get(eleszto_url)
    sleep(1)

    yes_class = "imgWrapper clickable"
    no_class = 'imgWrapper productSoldOut clickable'

    elems = browser.find_elements_by_xpath("//a[@class]")
    class_names = []
    no_class_counter = 0

    for elem in elems:
        class_names.append(elem.get_attribute("class"))
        if elem.get_attribute("class") == no_class:
            no_class_counter += 1
            
    if yes_class in class_names:
        van = True
        browser.close()
        send_email(text)

    if no_class_counter == 3:
        van = False
        print ("Nincs élesztő! {}".format(pd.datetime.now()))
        browser.close()
    
    return van
    



In [16]:
login = open('login.txt').readlines()
van = get_eleszto()
while not van:
    sched = scheduler(time, sleep)
    sched.enter(600, 1, get_eleszto)
    sched.run()
print ('Van élesztő!')


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="email"]"}
  (Session info: chrome=93.0.4577.82)


In [37]:
login = open('login.txt').readlines()
van = get_szallitasi_ido()
while not van:
    sched = scheduler(time, sleep)
    sched.enter(1, 1, get_szallitasi_ido)
    sched.run()
print ('Van szállítási idő!')

Nincs szállítási idő!
Nincs szállítási idő!
Nincs szállítási idő!


KeyboardInterrupt: 